In [245]:
# Используемые библы
import csv
import urllib.request
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import requests
from lxml import html
import numpy

In [ ]:
%%time
proxy_url = 'http://www.ip-adress.com/proxy_list/'
proxy_list = []

r = requests.get(proxy_url)
str_ = html.fromstring(r.content)
#/html/body/main/table/tbody/tr[1]/td[1]
result1 = str_.xpath("/html/body/main/table/tbody/tr/td[1]/a/text()")
result2 = str_.xpath("/html/body/main/table/tbody/tr/td[1]/text()")
result = []
for i in range(len(result1)):
    result.append(result1[i]+result2[i])
proxy_list = result

In [368]:
# сам парсер
def get_html(url):
    url_proxy = 'https://' + proxy_list[numpy.random.randint(50)]
    #print(url_proxy)
    #urllib.request.ProxyHandler(proxies = {'https': url_proxy})
    proxy = urllib.request.ProxyHandler(proxies = {'https': url_proxy})
    opener = urllib.request.build_opener(proxy)
    urllib.request.install_opener(opener)
    response = urllib.request.urlopen(url)
    return response.read()

def main_parse(html):
    soup = BeautifulSoup(html,"html.parser")
    table = soup.find('div', id ='listingResults')
    rows = table.find_all('a', itemprop = 'sameAs')
    links = []
    for row in rows:
        links.append(row['href'])
    return links

def sub_parse(html):
    soup = BeautifulSoup(html,"html.parser")
    try:
        table = soup.find('div', class_ ='b-listing-details')
        text = table.find('p').text
        #print(text)
        address = soup.find('div', class_ = 'e-listing-address').span.text
        #print(address)
        rows = table.find_all('div', class_ = 'e-table-column')
        price, price_meter, house_type, num_rooms, material, floor, wtf, all_area, kitchen_area, \
        live_area = 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.
        #print(price)
        z = []
        for row in rows:
            for i in row.find_all('ul'):
                z.append(i)
        #print(len(z))
        price = ''.join(z[0].find_all('li')[0].text[6:].strip().split()[:-1])
        print(price)
        price_meter = ''.join(z[0].find_all('li')[1].text[12:].strip().split()[:-1])
        house_type = z[0].find_all('li')[2].text.strip().split()[-1]
        num_rooms = z[0].find_all('li')[3].text.strip().split()[-1]
        material = z[0].find_all('li')[4].text.strip().split()[-1]
        floor = z[0].find_all('li')[5].text.strip().split()[-1].strip().split('/')[0]
        
        try:
            wtf = z[0].find_all('li')[6].text.strip().split()[-1]
        except:
            wtf = z[1].find_all('li')[0].text.strip().split()[-1]
        
        if z[1].find_all('li')[0].strong.text == 'Площадь:':
            all_area = z[1].find_all('li')[0].text[9:].strip().split()[0]
        else:
            all_area = z[1].find_all('li')[1].text[9:].strip().split()[0]
        
        if z[1].find_all('li')[1].strong.text == ' Площадь кухни (м²):':
            kitchen_area = z[1].find_all('li')[1].text[21:].strip().split()[0]
        else:
            kitchen_area = 'NA'
        
        if z[1].find_all('li')[2].strong.text == 'Жилая площадь (м²):':
            live_area = z[1].find_all('li')[2].text[20:].strip().split()[0]
        else:
            live_area = 'NA'
                        
    except:
        address = 0.
        text = 0.
        price, price_meter, house_type, num_rooms, material, floor, wtf, all_area, kitchen_area, \
        live_area = 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.
            
    return address, text, price, price_meter, house_type, num_rooms, material, floor, wtf, all_area, kitchen_area, \
        live_area

In [369]:
address_list, text_list, price_list, price_meter_list, house_type_list, num_rooms_list, \
material_list, floor_list, wtf_list, all_area_list, kitchen_area_list, live_area_list = [],[],[],[],[],[],[],[],[],[],[],[]
for i in tqdm(range(1,5)):
    BASE_URL = 'https://www.domofond.ru/prodazha-kvartiry-ufa-c1514?Page=' + str(i)
    a = main_parse(get_html(BASE_URL))
    for url in a:
        main_url  = 'https://www.domofond.ru' + url
        address, text, price, price_meter, house_type, num_rooms, material, floor, wtf, all_area, kitchen_area, \
        live_area = sub_parse(get_html(main_url))
        address_list.append(address)
        text_list.append(text)
        price_list.append(price)
        price_meter_list.append(price_meter)
        house_type_list.append(house_type)
        num_rooms_list.append(num_rooms)
        material_list.append(material)
        floor_list.append(floor)
        wtf_list.append(wtf)
        all_area_list.append(all_area)
        kitchen_area_list.append(kitchen_area)
        live_area_list.append(live_area)
        time.sleep(3)


  0%|                                                   | 0/10 [00:00<?, ?it/s]


URLError: <urlopen error Tunnel connection failed: 501 Tor is not an HTTP Proxy>

In [291]:
a[19]

'/4-komnatnaya-kvartira-na-prodazhu-ufa-187000177'

In [339]:
main_url  = 'https://www.domofond.ru' + a[19]
address, text, price, price_meter, house_type, num_rooms, material, floor, wtf, all_area, kitchen_area, \
live_area = sub_parse(get_html(main_url))

In [343]:
live_area

'NA'

In [317]:
help_url  = 'https://www.domofond.ru' + a[6]

In [318]:
w = help_parse(get_html(help_url))

In [327]:
w[1]

<ul>
<li>
<strong>Площадь:</strong>
36 м²                                            </li>
<li>
<strong> Площадь кухни (м²):</strong>
7 м²                                            </li>
<li>
<strong>Жилая площадь (м²):</strong>
24 м²                                            </li>
<li>
<strong>Дата обновления объявления:</strong>
04/08/2017                                            </li>
<li>
<strong>Дата публикации объявления:</strong>
12/06/2017                                            </li>
<li>
<strong>Номер в каталоге:</strong>
184762522                                            </li>
</ul>

In [332]:
w[1].find_all('li')[2].strong.text

'Жилая площадь (м²):'